In [1]:
!py -3.9 -m pip install keras

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [84]:
def permute(original, order):
    """Permute the bits of the original according to the specified order."""
    return ''.join(original[i - 1] for i in order)

def generate_subkeys(key):
    """Generate 8-bit subkeys for S-DES."""
    p10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
    p8 = [6, 3, 7, 4, 8, 5, 10, 9]

    # Apply the initial permutation (P10)
    key = permute(key, p10)

    # Split the key into two halves
    left_half = key[:5]
    right_half = key[5:]

    # Perform the first circular left shift on each half
    left_half = left_half[1:] + left_half[0]
    right_half = right_half[1:] + right_half[0]

    # Combine the halves
    combined_half = left_half + right_half

    # Apply the permutation (P8) to generate the first subkey
    subkey1 = permute(combined_half, p8)

    # Perform a second circular left shift on each half
    left_half = left_half[2:] + left_half[:2]
    right_half = right_half[2:] + right_half[:2]

    # Combine the halves
    combined_half = left_half + right_half

    # Apply the permutation (P8) to generate the second subkey
    subkey2 = permute(combined_half, p8)

    return subkey1, subkey2

def initial_permutation(block):
    """Perform the initial permutation (IP) on the 8-bit block."""
    ip = [2, 6, 3, 1, 4, 8, 5, 7]
    return permute(block, ip)

def inverse_permutation(block):
    """Perform the inverse permutation (IP^-1) on the 8-bit block."""
    ip_inverse = [4, 1, 3, 5, 7, 2, 8, 6]
    return permute(block, ip_inverse)

def expansion_permutation(block):
    """Perform the expansion permutation (EP) on the 4-bit block."""
    ep = [4, 1, 2, 3, 2, 3, 4, 1]
    return permute(block, ep)

def substitution_box(input_4bit):
    """Perform the substitution box operation (S-Box) on the 4-bit input."""
    s_box = [
        [[1, 0], [0, 3], [3, 2], [2, 1]],
        [[2, 1], [1, 0], [0, 3], [3, 2]],
        [[3, 2], [2, 1], [1, 0], [0, 3]],
        [[0, 3], [3, 2], [2, 1], [1, 0]]
    ]

    row = int(input_4bit[0] + input_4bit[3], 2)
    col = int(input_4bit[1:3], 2)

    return format(s_box[row][col][0], '02b') + format(s_box[row][col][1], '02b')

def f_function(right_half, subkey):
    """Perform the F function on the 4-bit right half."""
    # Expansion permutation
    expanded_right = expansion_permutation(right_half)

    # XOR with subkey
    expanded_right_xor_subkey = ''.join(str(int(a) ^ int(b)) for a, b in zip(expanded_right, subkey))

    # Substitution box (S-Box)
    s_box_output = substitution_box(expanded_right_xor_subkey)

    return s_box_output

def round_function(block, subkey):
    """Perform one round of S-DES."""
    left_half, right_half = block[:4], block[4:]

    # F function
    f_output = f_function(right_half, subkey)

    # XOR with left half
    new_right_half = ''.join(str(int(a) ^ int(b)) for a, b in zip(left_half, f_output))

    # Swap left and right halves
    new_left_half = right_half
    new_block = new_left_half + new_right_half

    return new_block

def sdes_encrypt(plaintext, key):
    """Encrypt the plaintext using S-DES."""
    key = format(int(key, 2), '010b')  # Convert key to 10-bit binary
    plaintext = format(int(plaintext, 2), '08b')  # Convert plaintext to 8-bit binary

    # Generate subkeys
    subkey1, subkey2 = generate_subkeys(key)

    # Initial permutation
    permuted_text = initial_permutation(plaintext)

    # Round 1
    permuted_text = round_function(permuted_text, subkey1)

    # Swap halves
    permuted_text = permuted_text[4:] + permuted_text[:4]

    # Round 2
    permuted_text = round_function(permuted_text, subkey2)

    # Inverse permutation
    ciphertext = inverse_permutation(permuted_text)

    return format(int(ciphertext, 2), '08b')

# Example usage
plaintext = '11011010'
key = '1010000010'

encrypted_text = sdes_encrypt(plaintext, key)
print("Plaintext:", plaintext)
print("Key:", key)
print("Encrypted Text:", encrypted_text)


Plaintext: 11011010
Key: 1010000010
Encrypted Text: 11100001


In [106]:
import random
import pandas as pd

N_TRAIN = 55000
N_VAL = 30000
N_TEST = 15000

M = 8
L = 10

def random_binary(length):
    return ''.join(random.choice('01') for _ in range(length))

def generate_dataset(encrypt, N, name):
    plains = []
    ciphers = []
    keys = []
    
    for _ in range(N):
        plain = random_binary(M)
        key = random_binary(L)
        cipher = encrypt(plain, key)
        
        plains.append(plain)
        ciphers.append(cipher)
        keys.append(key)
    
    df = pd.DataFrame({'Plaintext' : plains, 'Ciphertext' : ciphers, 'Key' : keys})
    df.to_csv(name, index=False)
        

In [183]:
# Training
generate_dataset(sdes_encrypt, N_TRAIN, 'S-DES-TRAIN.csv')

# Validation
generate_dataset(sdes_encrypt, N_VAL, 'S-DES-VAL.csv')

# Testing
generate_dataset(sdes_encrypt, N_TEST, 'S-DES-TEST.csv')

In [365]:
import tensorflow as tf
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

gpu_devices = tf.config.list_physical_devices('GPU')
details = tf.config.experimental.get_device_details(gpu_devices[0])
details.get('device_name', 'Unknown GPU')

'NVIDIA GeForce RTX 3050 Laptop GPU'

In [184]:
BATCH_SIZE = 32
LEARNING_RATE = 0.1
NUM_EPOCHS = 300

M = 8
L = 10

def embed(df):
  df['Input'] = df['Plaintext'].astype(str) + df['Ciphertext']
  df = df.drop(columns=['Plaintext', 'Ciphertext'])

  df['Key'] = df['Key'].apply(lambda x : np.array(list(map(int, list(x))), dtype=np.float32))
  df['Input'] = df['Input'].apply(lambda x : np.array(list(map(int, list(x))), dtype=np.float32))

  input = np.vstack(df['Input'].to_numpy())
  key = np.vstack(df['Key'].to_numpy())

  return input, key

dtype_mapping = {
    'Plaintext' : str,
    'Ciphertext' : str,
    'Key' : str
}
# Training
df_train = pd.read_csv('S-DES-TRAIN.csv', dtype=dtype_mapping)

# Validation
df_val = pd.read_csv('S-DES-VAL.csv', dtype=dtype_mapping)

# Testing
df_test = pd.read_csv('S-DES-TEST.csv', dtype=dtype_mapping)

train_input, train_key = embed(df_train)
val_input, val_key = embed(df_val)
test_input, test_key = embed(df_test)

In [411]:
seed_value = 42
tf.random.set_seed(seed_value)

initializer='truncated_normal'

class GLU(layers.Layer):
    def __init__(self, units):
        super(GLU, self).__init__()
        self.units = units
        self.output_layer = layers.Dense(2 * units, activation=None, trainable=True, kernel_initializer=initializer)

    def call(self, inputs):
        linear_output = self.output_layer(inputs)
        return tf.multiply(linear_output[:, :self.units], tf.keras.activations.sigmoid(linear_output[:, self.units:]))

class ResidualBlock(layers.Layer):
    def __init__(self, size):
        super(ResidualBlock, self).__init__()
        self.size = size
        
        activation=None
        
        self.input_layer = layers.Dense(16, activation=activation, trainable=True, kernel_initializer=initializer)
        
        self.bn1 = layers.BatchNormalization()
        self.linear1 = layers.Dense(size, activation=activation, trainable=True, kernel_initializer=initializer)
        self.bn2 = layers.BatchNormalization()
        self.linear2 = layers.Dense(size, activation=activation, trainable=True, kernel_initializer=initializer)
        self.bn3 = layers.BatchNormalization()
        self.linear3 = layers.Dense(size, activation=activation, trainable=True, kernel_initializer=initializer)
        self.bn4 = layers.BatchNormalization()
        self.linear4 = layers.Dense(size, activation=activation, trainable=True, kernel_initializer=initializer)
    
    def call(self, inputs):
        x = self.input_layer(inputs)
        
        x = self.bn1(x)
        x = tf.keras.activations.relu(x)
        x = self.linear1(x)
        x = self.bn2(x)
        x = tf.keras.activations.relu(x)
        x = self.linear2(x)
        
        y = self.bn3(x)
        y = tf.keras.activations.relu(y)
        y = self.linear3(y)
        y = self.bn4(y)
        y = tf.keras.activations.relu(y)
        y = self.linear4(y)
        
        z = x + y
        return z
        

class SDESModel(tf.keras.Model):
    def __init__(self, size):
        super(SDESModel, self).__init__()
        self.size = size
        
        self.block = ResidualBlock(size)
        self.glu = GLU(10)
        
        self.output_layer = layers.Dense(10, trainable=True, activation='relu', kernel_initializer=initializer)

    def call(self, inputs):
        x = self.block(inputs)
        y = self.glu(x)
        
        return self.output_layer(y)

In [412]:
def xnor_layer(tensor1, tensor2):
    # Convert tensors to boolean values
    bool_tensor1 = tf.math.greater_equal(tensor1, 0.5)  # Assuming values >= 0.5 are considered True
    bool_tensor2 = tf.math.greater_equal(tensor2, 0.5)

    # Perform logical AND and logical OR operations
    logical_and = tf.math.logical_and(bool_tensor1, bool_tensor2)
    logical_or = tf.math.logical_or(tf.math.logical_not(bool_tensor1), tf.math.logical_not(bool_tensor2))

    # Perform logical NOT operation
    xnor_result = tf.math.logical_not(tf.math.logical_or(logical_and, logical_or))

    # Convert boolean result back to float32
    xnor_result = tf.cast(xnor_result, tf.float32)

    return xnor_result

def BAP(predictions, targets, l):
  k = targets[:, l]
  k_pred = (predictions[:, l] >= 0.5).float()

  return tf.reduce_sum(xnor_layer(k, k_pred)) / len(df_test)

In [436]:
NUM_EPOCHS = 150
BATCH_SIZE = len(df_train)

initial_learning_rate = 0.1
final_learning_rate = 0.1
learning_rate_decay_factor = (final_learning_rate / initial_learning_rate)**(1/NUM_EPOCHS)
steps_per_epoch = int(len(df_train)/BATCH_SIZE)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=initial_learning_rate,
                decay_steps=steps_per_epoch,
                decay_rate=learning_rate_decay_factor,
                staircase=True)

In [437]:
model = SDESModel(128)

In [438]:
model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss='mean_squared_error',
    metrics=['accuracy'],
)

In [439]:
history = model.fit(train_input, train_key, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 0.4903 - accuracy: 0.0503
Epoch 2/150
1/1 [==============================] - 0s 57ms/step - loss: 900.7255 - accuracy: 0.1244
Epoch 3/150
1/1 [==============================] - 0s 53ms/step - loss: 0.4994 - accuracy: 0.5003
Epoch 4/150
1/1 [==============================] - 0s 46ms/step - loss: 0.4994 - accuracy: 0.5003
Epoch 5/150
1/1 [==============================] - 0s 50ms/step - loss: 0.4994 - accuracy: 0.5003
Epoch 6/150
1/1 [==============================] - 0s 37ms/step - loss: 0.4994 - accuracy: 0.5003
Epoch 7/150
1/1 [==============================] - 0s 47ms/step - loss: 0.4994 - accuracy: 0.5003
Epoch 8/150
1/1 [==============================] - 0s 41ms/step - loss: 0.4994 - accuracy: 0.5003
Epoch 9/150
1/1 [==============================] - 0s 42ms/step - loss: 0.4994 - accuracy: 0.5003
Epoch 10/150
1/1 [==============================] - 0s 42ms/step - loss: 0.4994 - accuracy: 0.5003
Epoch 11/150
1/1 [=

In [378]:
test_key[0]

array([1., 1., 0., 1., 1., 0., 1., 1., 0., 1.], dtype=float32)

In [379]:
test_input[0]

array([1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1.],
      dtype=float32)

In [380]:
model.predict(test_input)[0]

469/469 [==============================] - 2s 3ms/step


array([0.1786572 , 0.17420608, 0.18691818, 0.20201515, 0.17594269,
       0.19635528, 0.18544662, 0.18409753, 0.20407243, 0.18878058],
      dtype=float32)